In [5]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity

In [6]:
customers_df = pd.read_csv('Customers.csv')
products_df = pd.read_csv("Products.csv")
transactions_df = pd.read_csv('Transactions.csv')

In [13]:
# Merge all features
features = pd.merge(transaction_behavior, last_transaction, on='CustomerID')
features = pd.merge(features, product_diversity, on='CustomerID')
features = pd.merge(features, price_sensitivity, on='CustomerID')
features = pd.merge(features, favorite_day, on='CustomerID')
features = pd.merge(features, category_features, on='CustomerID')

# Add Region and Tenure
features = pd.merge(features, customers[['CustomerID', 'Region']], on='CustomerID')
features = pd.merge(features, data[['CustomerID', 'Tenure']].drop_duplicates(), on='CustomerID')
features

,CustomerID,TotalSpending,AvgSpending,StdSpending,TotalQuantity,AvgQuantity,StdQuantity,TransactionFrequency,AvgPricePaid,LastTransactionDate,...,Quantity_Books,Quantity_Clothing,Quantity_Electronics,Quantity_Home Decor,Price_product_Books,Price_product_Clothing,Price_product_Electronics,Price_product_Home Decor,Region,Tenure
0,C0001,3354.52,670.904000,456.643861,12,2.400000,0.547723,5,278.334000,2024-11-02 17:04:16,...,2,0,7,3,57.30,0.000,398.943333,137.540000,South America,932
1,C0002,1862.74,465.685000,219.519169,10,2.500000,1.000000,4,208.920000,2024-12-03 01:41:41,...,0,4,0,6,0.00,256.365,0.000000,161.475000,Asia,1079
2,C0003,2725.38,681.345000,559.276543,14,3.500000,0.577350,4,195.707500,2024-08-24 18:54:04,...,0,4,4,6,0.00,30.590,346.300000,202.970000,South America,326
3,C0004,5354.88,669.360000,325.386829,23,2.875000,1.125992,8,240.636250,2024-12-23 14:13:52,...,8,0,6,9,268.65,0.000,206.180000,235.593333,South America,841
4,C0005,2034.24,678.080000,310.820746,7,2.333333,0.577350,3,291.603333,2024-11-04 00:30:22,...,0,0,4,3,0.00,0.000,295.095000,284.620000,Asia,896
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,C0196,4982.88,1245.720000,573.093969,12,3.000000,1.414214,4,416.992500,2024-12-15 03:43:35,...,3,4,0,5,436.89,396.340,0.000000,417.370000,Europe,965
195,C0197,1928.65,642.883333,321.163312,9,3.000000,1.000000,3,227.056667,2024-12-27 18:20:31,...,0,0,6,3,0.00,0.000,171.630000,337.910000,Europe,678
196,C0198,931.83,465.915000,620.733688,3,1.500000,0.707107,2,239.705000,2024-10-04 18:31:12,...,0,2,1,0,0.00,452.420,26.990000,0.000000,Europe,1065
197,C0199,1979.28,494.820000,256.090945,9,2.250000,1.258306,4,250.610000,2024-10-26 00:01:58,...,0,0,3,6,0.00,0.000,223.765000,277.455000,Europe,786


In [17]:
features.columns

Index(['CustomerID', 'TotalSpending', 'AvgSpending', 'StdSpending',
       'TotalQuantity', 'AvgQuantity', 'StdQuantity', 'TransactionFrequency',
       'AvgPricePaid', 'LastTransactionDate', 'DaysSinceLastTransaction',
       'UniqueProducts', 'UniqueCategories', 'AvgPriceSensitivity',
       'TotalValue_Books', 'TotalValue_Clothing', 'TotalValue_Electronics',
       'TotalValue_Home Decor', 'Quantity_Books', 'Quantity_Clothing',
       'Quantity_Electronics', 'Quantity_Home Decor', 'Price_product_Books',
       'Price_product_Clothing', 'Price_product_Electronics',
       'Price_product_Home Decor', 'Tenure', 'Region_Asia', 'Region_Europe',
       'Region_North America', 'Region_South America',
       'FavoriteTransactionDay_0', 'FavoriteTransactionDay_1',
       'FavoriteTransactionDay_2', 'FavoriteTransactionDay_3',
       'FavoriteTransactionDay_4', 'FavoriteTransactionDay_5',
       'FavoriteTransactionDay_6'],
      dtype='object')

In [20]:
# Encode categorical features (Region, FavoriteTransactionDay)
# features = pd.get_dummies(features, columns=['Region', 'FavoriteTransactionDay_0'])

# Normalize numerical features
numerical_cols = features.select_dtypes(include=['int64', 'float64']).columns.tolist()
print(numerical_cols)
# numerical_cols.remove('CustomerID')  # Exclude CustomerID
scaler = StandardScaler()
features[numerical_cols] = scaler.fit_transform(features[numerical_cols])

['TotalSpending', 'AvgSpending', 'StdSpending', 'TotalQuantity', 'AvgQuantity', 'StdQuantity', 'TransactionFrequency', 'AvgPricePaid', 'DaysSinceLastTransaction', 'UniqueProducts', 'UniqueCategories', 'AvgPriceSensitivity', 'TotalValue_Books', 'TotalValue_Clothing', 'TotalValue_Electronics', 'TotalValue_Home Decor', 'Quantity_Books', 'Quantity_Clothing', 'Quantity_Electronics', 'Quantity_Home Decor', 'Price_product_Books', 'Price_product_Clothing', 'Price_product_Electronics', 'Price_product_Home Decor', 'Tenure']


In [24]:
from sklearn.impute import SimpleImputer
import numpy as np

# Separate numeric and non-numeric columns
numeric_cols = features.select_dtypes(include=['int64', 'float64']).columns.tolist()
non_numeric_cols = features.select_dtypes(exclude=['int64', 'float64']).columns.tolist()

# Impute missing values in numeric columns
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
features[numeric_cols] = imp.fit_transform(features[numeric_cols])

# Recombine numeric and non-numeric columns
features = features[numeric_cols + non_numeric_cols]

# Normalize numerical features
scaler = StandardScaler()
features[numeric_cols] = scaler.fit_transform(features[numeric_cols])

# Calculate cosine similarity
similarity_matrix = cosine_similarity(features[numeric_cols])

# Generate recommendations for first 20 customers
lookalike_map = {}
for i in range(20):
    cust_id = features.iloc[i]['CustomerID']
    sim_scores = list(enumerate(similarity_matrix[i]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:4]  # Exclude self
    lookalike_map[cust_id] = [
        (features.iloc[j]['CustomerID'], round(score, 3)) 
        for j, score in sim_scores
    ]

# Save to CSV
lookalike_df = pd.DataFrame.from_dict(lookalike_map, orient='index')
lookalike_df.to_csv('Lookalike.csv', header=False)

In [25]:
similarity_matrix

array([[ 1.        , -0.04297444,  0.42349381, ...,  0.07607923,
         0.30372857, -0.55044909],
       [-0.04297444,  1.        ,  0.10892565, ...,  0.557428  ,
         0.63422572, -0.23825253],
       [ 0.42349381,  0.10892565,  1.        , ..., -0.06907747,
         0.3023494 , -0.25744848],
       ...,
       [ 0.07607923,  0.557428  , -0.06907747, ...,  1.        ,
         0.2398585 , -0.34765839],
       [ 0.30372857,  0.63422572,  0.3023494 , ...,  0.2398585 ,
         1.        , -0.64508401],
       [-0.55044909, -0.23825253, -0.25744848, ..., -0.34765839,
        -0.64508401,  1.        ]], shape=(199, 199))